# Climate and Soil

This script outputs the satellite-based rasters into the Google Earth Engine Cloud.

Inputs:


Outputs:




In [2]:
import ee
import geemap
from gee_0_utils import *

initialize()
config = ProjectConfig()
roi = config.roi
data_folder = config.data_folder

## Soil Data from SoilGrids

250m resolution

        - Bulk Density
        - Cation Exchange Capacity
        - Clay Content
        - Coarse fragments (> 2 mm)
        - Nitrogen
        - Organic Carbon Density
        - Soil Organic Carbon Stock
        - pH
        - Sand Content
        - Soil Organic Carbon
    All averaged from 0-30cm depth and converted to the correct units.

In [ ]:
bdod = ee.Image("projects/soilgrids-isric/bdod_mean").clip(roi) # bulk density
cec = ee.Image("projects/soilgrids-isric/cec_mean") # cation exchange capacity
clay = ee.Image("projects/soilgrids-isric/clay_mean").clip(roi)
cfvo = ee.Image("projects/soilgrids-isric/cfvo_mean").clip(roi) # coarse fragments
nitro = ee.Image("projects/soilgrids-isric/nitrogen_mean").clip(roi)
ocd = ee.Image("projects/soilgrids-isric/ocd_mean").clip(roi) # organic carbon density
ocs = ee.Image("projects/soilgrids-isric/ocs_mean").clip(roi) # soil organic carbon stock
phh2o = ee.Image("projects/soilgrids-isric/phh2o_mean").clip(roi)
sand = ee.Image("projects/soilgrids-isric/sand_mean").clip(roi)
soc = ee.Image("projects/soilgrids-isric/soc_mean").clip(roi) # soil organic carbon

# Function to select and calculate weighted mean for specific depth bands
def weighted_means(image, name):
    depths = ['0-5cm', '5-15cm', '15-30cm']
    weights = [1, 2, 3]    
    # Select the bands for each depth
    weighted_bands = image.select([f'.*_{depth}_mean' for depth in depths]).multiply(ee.Image.constant(weights))
    # Sum the weighted bands
    weighted_mean = weighted_bands.reduce(ee.Reducer.sum()).divide(6)
    return weighted_mean

# Unit conversions
bdod = bdod.multiply(10)  # cg/cm³ to kg/m³
nitro = nitro.divide(100)  # cg/kg to g/kg
soc = soc.divide(10)  # dg/kg to g/kg
cec = cec.divide(10)  # mmol(c)/kg to cmol/kg

# Convert nitrogen and soil organic carbon to g/m²
nitro = weighted_means(nitro.multiply(bdod), "nitro")  # g/kg to g/m³
soc = weighted_means(soc.multiply(bdod), "soc")  # g/kg to g/m³
cec = weighted_means(cec.multiply(bdod), "cec")  # cmol/kg to cmol/kg

# Apply unit conversions and depth aggregation to other soil properties
clay = weighted_means(clay.divide(10), "clay")  # g/kg (‰) to g/100g (%)
cfvo = weighted_means(cfvo.divide(10), "cfvo")  # cm³/dm³ (‰) to cm³/100cm³ (%)
ocd = weighted_means(ocd.divide(10), "ocd")  # hg/m³ to kg/m³
ocs = ocs.divide(10)  # hg/m² to kg/m²
phh2o = weighted_means(phh2o.divide(10), "phh2o")  # pH x 10 to pH
sand = weighted_means(sand.divide(10), "sand")  # g/kg (‰) to g/100g (%)

# Combine all soil properties into a single image
soil_properties = bdod.addBands([cec, clay, cfvo, nitro, ocd, ocs, phh2o, sand, soc]).float()

# Export the final image
export_image(soil_properties, "soilgrids", region = roi, scale = 250)

## TerraClim Data

    Calculated yearly metrics.
        Summed (data shown as monthly totals):
        - Soil Moisture (mm)
        - Precipitation (mm)
        - Evapotranspiration (mm)
        - Climate Water Deficit (mm)

        Averaged (data shown as monthly averages):
        - Solar Radiation (W/m^2)
        - Temperature (C)
        - Vapour Pressure Deficit (kPa)
        - Palmer Drought Severity Index (PDSI)

https://gee-community-catalog.org/projects/terraclim/

<!-- # Terraclim
  https://developers.google.com/earth-engine/datasets/catalog/IDAHO_EPSCOR_TERRACLIMATE
Bring temperature and precipitation and calculate seasonality -->

<!-- ## Seasonality index

Walsh and Lawler 1981 -->

In [6]:
terraclim = (
    ee.ImageCollection("IDAHO_EPSCOR/TERRACLIMATE")
    .filterDate("1985-01-01", "2019-12-31")
    .map(lambda image: image.reduceResolution(ee.Reducer.median(), bestEffort = True, maxPixels = 1024)
    .reproject(crs = "EPSG:4326", scale = 10000))
)

# Function to calculate yearly metrics with scaling applied after filtering
def calculate_yearly_metrics(year):

    def scale_and_aggregate(var, reducer):
        data = terraclim.select(var).filter(ee.Filter.calendarRange(year, year, "year"))
        if var in ["tmmn", "tmmx", "soil", "aet", "def", "srad"]:
            data = data.map(lambda img: img.multiply(0.1).toInt16())
        if var in ["pdsi", "vpd"]:
            data = data.map(lambda img: img.multiply(0.01).float())
        return data.reduce(reducer).rename(f"{var}_{year}")

    # Define which variables are processed with sum or mean reducers
    sum_vars = ["soil", "pr", "aet", "def"]
    mean_vars = ["srad", "vpd", "pdsi"]

    # Aggregate sum variables (radiation, soil, precipitation)
    processed_vars = {var: scale_and_aggregate(var, ee.Reducer.sum()) for var in sum_vars}
    
    # Aggregate mean temperature (average of max and min)
    maxtemp = scale_and_aggregate("tmmx", ee.Reducer.mean())
    mintemp = scale_and_aggregate("tmmn", ee.Reducer.mean())
    processed_vars["temp"] = maxtemp.addBands(mintemp).reduce(ee.Reducer.mean()).float().rename(f"temp_{year}")
    
    # Aggregate other mean variables
    processed_vars.update({var: scale_and_aggregate(var, ee.Reducer.mean()) for var in mean_vars})

    # Calculate Seasonality Index (SI)
    mean_prec = processed_vars["pr"].divide(12),
    deviations = terraclim.select("pr").filter(ee.Filter.calendarRange(year, year, "year")) \
        .map(lambda img: img.multiply(0.1))\
        .map(lambda month: month.subtract(mean_prec).abs())
    si_band = deviations.reduce(ee.Reducer.sum()).divide(processed_vars["pr"]).float().rename(f"si_{year}")

    return ee.Image.cat([*processed_vars.values(), si_band])

# Create a dictionary for variables with filtering and scaling applied after
vars = {var: terraclim.select(var) for var in ["tmmx", "tmmn", "srad", "vpd", "soil", "aet", "pr", "def", "pdsi"]}

# Calculate yearly metrics and combine into a single image
yearly_metrics = ee.Image.cat([calculate_yearly_metrics(year) for year in config.range_1985_2019])

# Function to calculate the mean for a given band pattern
def calculate_mean(band_pattern, new_name):
    return yearly_metrics.select(band_pattern).reduce(ee.Reducer.mean()).rename(new_name)

# Calculate the mean across all years for the desired variables
mean_metrics = {name: calculate_mean(f".*{name}.*", f"mean_{name}") \
                for name in ["pr", "srad", "temp", "vpd", "soil", "aet", "si", "def", "pdsi"]}

# Combine the mean layers into a single image
yearly_terraclim = ee.Image.cat([yearly_metrics, *mean_metrics.values()])

# Export the final image
export_image(yearly_terraclim, "yearly_terraclim3", region = roi, scale = 10000)



### Climate
https://developers.google.com/earth-engine/datasets/catalog/NASA_GDDP-CMIP6

- huss: Near-surface relative humidity (%)
- pr: Mean of daily precipitation rate (kg m-2 s-1)
- rsds: Surface downwelling shortwave radiation (W m-2)
- tas: Daily near-surface air temperature (K)

In [ ]:
# incompleteSims=['BCC-CSM2-MR','CESM2', 'CESM2-WACCM','IITM-ESM','IPSL-CM6A-LR','KIOST-ESM',
    #   'MIROC6','NESM3','NorESM2-LM', 'TaiESM1']

climate_CMIP6 = (ee.ImageCollection("NASA/GDDP-CMIP6")
                     .map(lambda image: image.select(["hurs", "pr", "rsds", "tas"])))

def calculate_yearly_metrics(year):
    climate_year = climate_CMIP6.filterDate(f"{year}-01-01", f"{year}-12-31").mean()
    climate_year = climate_year.rename([f"{band}_{year}" for band in climate_year.bandNames().getInfo()])
    return climate_year

years = list(range(1985, 2051))

yearly_metrics = ee.Image.cat([calculate_yearly_metrics(year) for year in years])

# Calculate the mean across all years for the desired variables
mean_metrics = {name: calculate_mean(f".*{name}.*", f"mean_{name}") \
                for name in ["hurs", "pr", "rsds", "tas"]}

# Combine the mean layers into a single image
yearly_cmip6 = ee.Image.cat([yearly_metrics, *mean_metrics.values()])


export_image(yearly_cmip6, "yearly_cmip6", region = roi, scale = 10000)

# map = geemap.Map()
# map.addLayer(yearly_metrics.select("tas_1985"), {"min": 250, "max": 320, "palette": ["blue", "green", "yellow", "red"]}, "Temperature 1985")
# map